In [1]:
import numpy as np
import tensorflow as tf
import gzip
import pickle
import sys
from ddm.run_split import SplitMnistGenerator
from ddm.alg.cla_models_multihead import MFVI_IBP_NN, Vanilla_NN
#from ddm.alg.vcl import run_vcl, run_vcl_ibp
from copy import deepcopy

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# IBP network

In [ ]:
hidden_size = [50,50]
batch_size = 512
no_epochs = 100
alpha0 = 50.0
tau0=1.0 # initial temperature
ANNEAL_RATE=0.000
MIN_TEMP=0.5

# Run vanilla VCL
tf.set_random_seed(12)
np.random.seed(1)

coreset_size = 0
data_gen = SplitMnistGenerator()
single_head=False
in_dim, out_dim = data_gen.get_dims()
x_testsets, y_testsets = [], []
for task_id in range(data_gen.max_iter):
    x_train, y_train, x_test, y_test = data_gen.next_task()
    x_testsets.append(x_test)
    y_testsets.append(y_test)

    # Set the readout head to train
    head = 0 if single_head else task_id
    bsize = x_train.shape[0] if (batch_size is None) else batch_size
    
    # Train network with maximum likelihood to initialize first model
    if task_id == 0:
        ml_model = Vanilla_NN(in_dim, hidden_size, out_dim, x_train.shape[0])
        ml_model.train(x_train, y_train, task_id, no_epochs, bsize)
        mf_weights = ml_model.get_weights()
        mf_variances = None
        mf_betas = None
        ml_model.close_session()

    # Train on non-coreset data
    mf_model = MFVI_IBP_NN(in_dim, hidden_size, out_dim, x_train.shape[0], prev_means=mf_weights, 
                           prev_log_variances=mf_variances, prev_betas=mf_betas,alpha0=alpha0,
                           learning_rate=0.01, temp_prior=1.0, no_pred_samples=100)
    mf_model.train(x_train, y_train, head, no_epochs, bsize, tau0=tau0, 
                   anneal_rate=ANNEAL_RATE, min_temp=MIN_TEMP)
    mf_weights, mf_variances, mf_betas = mf_model.get_weights()

    # Incorporate coreset data and make prediction
#     x_coresets = []
#     y_coresets = []
#     acc = get_scores(mf_model, x_testsets, y_testsets, x_coresets, y_coresets, hidden_size, no_epochs, single_head, batch_size)
#     all_acc = concatenate_results(acc, all_acc)

    mf_model.close_session()

Epoch: 0001 cost= 0.326666718
Epoch: 0006 cost= 0.002547519
Epoch: 0011 cost= 0.001163158
Epoch: 0016 cost= 0.000456997
Epoch: 0021 cost= 0.000236349
Epoch: 0026 cost= 0.000126062
Epoch: 0031 cost= 0.000080582
Epoch: 0036 cost= 0.000053394
Epoch: 0041 cost= 0.000039299
Epoch: 0046 cost= 0.000029185
Epoch: 0051 cost= 0.000023426
Epoch: 0056 cost= 0.000018092
Epoch: 0061 cost= 0.000014716
Epoch: 0066 cost= 0.000012215
Epoch: 0071 cost= 0.000010244
Epoch: 0076 cost= 0.000008630
Epoch: 0081 cost= 0.000007770
Epoch: 0086 cost= 0.000006347
Epoch: 0091 cost= 0.000005492
Epoch: 0096 cost= 0.000004839
Optimization Finished!
logpis: (100, 784, 50)
logpis: (100, 1, 50)
act: (100, ?, 50)
logpis: (100, 50, 50)
logpis: (100, 1, 50)
act: (100, ?, 50)
logpis: <unknown>
logpis: <unknown>
logpis: (100, 784, 50)
logpis: (100, 784, 50)
log_post: (100, 784, 50)
log_prior: (100, 784, 50)
log_sample: (100, 784, 50)
mean: (784, 50)
logpis: (100, 784, 50)
logpis: (100, 784, 50)
log_post: (100, 784, 50)
log_pri